# This notebook creates the MIRI Imager boresight offsets table for the PRD #

This version has been revised July 2 2022 to use coron-specific boresight offsets in the coron regions.
Further revision Oct 17 2022 reads these offsets from a new FLT-2 FITS distortion file rather than a temporary file.

In [ ]:
import numpy as np
import os
import getpass
import socket
import pdb as pdb
from astropy.modeling import models
from asdf import AsdfFile
from jwst import datamodels
from jwst.assign_wcs import miri
from astropy.io import fits
import datetime
from astropy.io import ascii
import math

rpd=math.pi/180.

Import the MIRI coordinates code from https://github.com/STScI-MIRI/miricoord and ensure that it is on the PYTHONPATH.  Also ensure that the output data directory is set:<br>
setenv MIRICOORD_DATA_DIR /YourLocalPathToData/ (this is where io will happen)

In [ ]:
# data_dir=os.path.expandvars('$MIRICOORD_DATA_DIR')
data_dir="./"#"/Users/jaguilar/Projects/jwst_calibration/boresight_offsets/f1550c_f1000w/output/"

Import the MIRI standalone tools

In [ ]:
import miricoord.imager.mirim_tools as mt
mt.set_toolversion('flt3')
mt.version()

Import the MIRI pipeline tools

The version mismatch is OK because you probably haven't generated new distortion files yet but the distortion model is the same

In [ ]:
import miricoord.imager.mirim_pipetools as mpt
mpt.set_toolversion('flt2')
mpt.version()

<br>

Import the pysiaf tools for working with the SIAF, and read the MIRI apertures

In [ ]:
import pysiaf

In [ ]:
siaf = pysiaf.Siaf('MIRI') 

In [ ]:
siafversion=pysiaf.JWST_PRD_VERSION
print(siafversion)

<BR>

Load up the MIRI Imager distortion model (using F770W since we're COMPUTING offsets rather than using them)

In [ ]:
model=mpt.xytov2v3model('F770W')

<BR>

Point to the IDT-delivered CDP file that we're using. This should be the one generated by `miricoord/miricoord/imager/makeboresight/UpdateFitsRef.ipynb`

In [ ]:
cdpfile=mt.get_fitsreffile()
cdpfile

In [ ]:
hdul = fits.open(cdpfile)

In [ ]:
tbdata = hdul['Boresight offsets'].data

In [ ]:
#print(tbdata.columns)

In [ ]:
allfilters=tbdata.field('filter')

In [ ]:
nfilter=len(allfilters)

In [ ]:
tbdata[10]

In [ ]:
# Read coron-specific boresight offsets from the imager reference file
coron=fits.open(cdpfile)

bore1065=coron['BoresightCORON1065'].data
bore1140=coron['BoresightCORON1140'].data
bore1550=coron['BoresightCORON1550'].data
borelyot=coron['BoresightCORONLYOT'].data

In [ ]:
for row in bore1065:
    print(f"{row[0]}\t{row[1]:+0.3f} {row[2]:+0.3f}")

Define two output files: One for TA boresights, one for science boresights

In [ ]:
taoutfile=data_dir+'miri_boresight_ta.txt'
scioutfile=data_dir+'miri_boresight_sci.txt'

Log information about when the files were run, with what inputs

In [ ]:
now=datetime.datetime.now()

In [ ]:
print('#',now.isoformat(),file=open(taoutfile,"w"))
print('#',now.isoformat(),file=open(scioutfile,"w"))
print('# Created by user',getpass.getuser(),'at',socket.gethostname(),file=open(taoutfile,"a"))
print('# Created by user',getpass.getuser(),'at',socket.gethostname(),file=open(scioutfile,"a"))
print('# Boresight pixel file:',cdpfile,file=open(taoutfile,"a"))
print('# Boresight pixel file:',cdpfile,file=open(scioutfile,"a"))
print('# Previous SIAF file:',siafversion,file=open(taoutfile,"a"))
print('# Previous SIAF file:',siafversion,file=open(scioutfile,"a"))
print('# ')
print('# Offsets (arcsec) are set so that they should be ADDED to the commanded position',file=open(taoutfile,"a"))
print('# Offsets (arcsec) are set so that they should be ADDED to the commanded position',file=open(scioutfile,"a"))

Print a header line about the column entries

In [ ]:
print("{:20} {:10} {:10} {:10}".format('# AperName','TAFilter','dv2','dv3'), file=open(taoutfile,"a"))
print("{:20} {:10} {:10} {:10}".format('# AperName','SciFilter','dv2','dv3'), file=open(scioutfile,"a"))

<BR>

Set up the desired sign of the outputs.  We want APT to ADD both the TA filter and science filter corrections, therefore we need to incorporate the negative sign on the TA filter offset into our reported values

In [ ]:
# original
tasign=-1
scisign=+1



<BR>

We will in each case do the conversion from pixel boresight offset to v2/v3 offset.  We will use the location of the TA aperture for MRS and LRS slit (because these science apertures don't have explicit pixel locations on the imager), and the science aperture for everything else.  Offsets are given in v2/v3 to avoid ambiguities with the Ideal coordinate systems.

<BR>

In [ ]:
#F560W to F2300C:
# TA file:  0.0082     -0.0074 
# Sci file: 0.1659     0.2187  
# Total: 0.1741 0.2113

In [ ]:
v2ref,v3ref=-389.524965,-337.768837

In [ ]:
mt.v2v3toIdeal(v2ref+0.1741,v3ref+0.2113,'MIRIM_MASKLYOT')

In [ ]:
# A note on usage: OPGS will use the boresight shifts as corrections to the MEASURED source location during TA
# when determining the offset to use to go to the science aperture.  I've confirmed this using program 1050
# Observation 9, which does TA in FND before going to the undithered MRS reference point.  This offset is
# commanded with an SCSCAMMAIN in guider ideal coordinates.  The offset in FGS1 guider ideal coordinates is given
# as -21.348, -0.330.  The expected offset just between SIAF ref locations is -21.344, -0.359.  The expected offset
# if the TA location had the appropriate boresight offsets from PRD added to it is -21.348, -0.330.  

## Do the MRS case ##

In [ ]:
print("",file=open(taoutfile,"a"))
print("",file=open(scioutfile,"a"))

# Loop over each TA filter computing offsets
for i in range(0,nfilter):
    thisentry=siaf['MIRIM_TAMRS']
    xdet,ydet=thisentry.XDetRef-1,thisentry.YDetRef-1 # Define the location in 0-indexed pixels (subtract 1 from SIAF)
    v2ref,v3ref=thisentry.V2Ref,thisentry.V3Ref
    x,y=xdet+tbdata[i].field('col_offset'),ydet+tbdata[i].field('row_offset')
    v2,v3=model(x,y)
    dv2,dv3=v2-v2ref,v3-v3ref
    # Print the negative of the offsets so that they can be added to the commanded position
    print("{val1:20} {val2:10} {0:<10.4f} {1:<10.4f}".format(tasign*dv2,tasign*dv3,val1='MIRIFU*',val2=tbdata[i].field('filter')),file=open(taoutfile,"a"))
    if (i == 2):
        dv2_save=tasign*dv2
        dv3_save=tasign*dv3
        
# Loop over each science filter (for MRS only one option)
print("{val1:20} {val2:10} {0:<10.4f} {1:<10.4f}".format(0,0,val1='MIRIFU*',val2='ALL'),file=open(scioutfile,"a"))

In [ ]:
# Sanity check
# If doing TA in F1000W, after TA the source would be centered at 
x0,y0 = xdet,ydet # In F1000W
# In v2,v3 the source is at
v2,v3=mt.xytov2v3(xdet,ydet,'F1000W')
# So in F770W the source is currently at
xstart,ystart=mt.v2v3toxy(v2,v3,'F770W')
# OPGS adds the boresights to SIAF location to pretend source is at
v2temp,v3temp = v2ref+dv2_save, v3ref+dv3_save
# It wants to go to v2ref, v3ref and will apply an offset to get there
# Figure out what this is using any reference filter for deltas:
x1,y1=mt.v2v3toxy(v2temp,v3temp,'F770W')
x2,y2=mt.v2v3toxy(v2ref,v3ref,'F770W')
dx=x2-x1
dy=y2-y1

print('Should be at ',xdet,ydet)
print('Actually at ',xstart[0]+dx[0],ystart[0]+dy[0])

<BR>

## Do the LRS slit case ##

In [ ]:
print("",file=open(taoutfile,"a"))
print("",file=open(scioutfile,"a"))

# Loop over each TA filter computing offsets AT THE SCIENCE APERTURE LOCATION
for i in range(0,nfilter):
    thisentry=siaf['MIRIM_TALRS']
    xdet,ydet=thisentry.XDetRef-1,thisentry.YDetRef-1 # Define the location in 0-indexed pixels (subtract 1 from SIAF)
    v2ref,v3ref=thisentry.V2Ref,thisentry.V3Ref
    x,y=xdet+tbdata[i].field('col_offset'),ydet+tbdata[i].field('row_offset')
    v2,v3=model(x,y)
    dv2,dv3=v2-v2ref,v3-v3ref
    # Print the value of the offsets so that they can be added to the commanded position
    print("{val1:20} {val2:10} {0:<10.4f} {1:<10.4f}".format(tasign*dv2,tasign*dv3,val1='MIRIM_SLIT',val2=tbdata[i].field('filter')),file=open(taoutfile,"a"))
    
# Loop over each science filter (for LRS only one option)
print("{val1:20} {val2:10} {0:<10.4f} {1:<10.4f}".format(0,0,val1='MIRIM_SLIT',val2='ALL'),file=open(scioutfile,"a"))

<br>

## Do the LRS slitless case ##

In [ ]:
print("",file=open(taoutfile,"a"))
print("",file=open(scioutfile,"a"))

# Loop over each TA filter computing offsets
for i in range(0,nfilter):
    thisentry=siaf['MIRIM_SLITLESSPRISM']
    xdet,ydet=thisentry.XDetRef-1,thisentry.YDetRef-1 # Define the location in 0-indexed pixels (subtract 1 from SIAF)
    v2ref,v3ref=thisentry.V2Ref,thisentry.V3Ref
    x,y=xdet+tbdata[i].field('col_offset'),ydet+tbdata[i].field('row_offset')
    v2,v3=model(x,y)
    dv2,dv3=v2-v2ref,v3-v3ref
    # Print the value of the offsets so that they can be added to the commanded position
    print("{val1:20} {val2:10} {0:<10.4f} {1:<10.4f}".format(tasign*dv2,tasign*dv3,val1='MIRIM_SLITLESSPRISM',val2=tbdata[i].field('filter')),file=open(taoutfile,"a"))
    
# Loop over each science filter (for LRS only one option)
print("{val1:20} {val2:10} {0:<10.4f} {1:<10.4f}".format(0,0,val1='MIRIM_SLITLESSPRISM',val2='ALL'),file=open(scioutfile,"a"))

<BR>

## Do the coronagraphs ##

Note that we use the 'MASK' apertures since these are the ones that drive the actual pointing; the 'CORON' apertures are just convenient for display purposes.  We also need to use coronagraph-specific boresight
offsets instead of the general offsets.  In July 2022, also use special-commanding results to update the SIAF center locations.

As a result we'll do each coronagraph individually.

# Read in previous boresight offsets tables
prevscifile=os.path.join(os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd()))),'data/boresight/miri_boresight_sci-PRDOPSSOC-051.txt')
prevtafile=os.path.join(os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd()))),'data/boresight/miri_boresight_ta-PRDOPSSOC-051.txt')

prevsci=ascii.read(prevscifile)
prevta=ascii.read(prevtafile)

## Do the Coron1065 ##

In [ ]:
# Current SIAF location
thisaper='MIRIM_MASK1065'

# Which boresight table are we using?
thisbore=bore1065.copy()

In [ ]:
# Commissioning special offsets in Ideal coordinates
specialx=0.
specialy=0.

# Pierre additional tweaks in pixel coordinates
pierrex=-0.
pierrey=0.

In [ ]:
# Assume OPGS did the right thing based on analysis of Visit files
# Source started at intended position in the FND filter
xold,yold=siaf[thisaper].XDetRef-1,siaf[thisaper].YDetRef-1

# According to SIAF, it would be this v2,v3 reference point
v2oldref,v3oldref=siaf[thisaper].V2Ref,siaf[thisaper].V3Ref

print('Old SIAF reference point: ',xold+1,yold+1)# 1-indexed
print('Old SIAF reference point: ',v2oldref,v3oldref)

In [ ]:
# When we swapped to Coron filter, where did it go for real?
# Subtract difference between FND and Coron filters in new measurements
xbase,ybase = xold-thisbore[14][1], yold-thisbore[14][2]

# What was the boresight shift APPLIED by OPGS?
x1, y1 = xbase-(tbdata[10][1]-tbdata[14][1]), ybase-(tbdata[10][2]-tbdata[14][2])

# Special commanding was added in Ideal coordinates, convert to dv2,dv3
# Special commanding moves the SOURCE
v2old,v3old=mt.xytov2v3(x1,y1,'F770W') # Temporary location, filter doesn't matter
angle,parity=siaf[thisaper].V3IdlYAngle,siaf[thisaper].VIdlParity
dv2sp = parity*specialx*math.cos(angle*rpd) + specialy*math.sin(angle*rpd)
dv3sp = - parity*specialx*math.sin(angle*rpd) + specialy*math.cos(angle*rpd)
# Effective location after special commanding
tempx,tempy = mt.v2v3toxy(v2old,v3old,'F770W')
x2, y2 = mt.v2v3toxy(v2old+dv2sp,v3old+dv3sp,'F770W')
# Take out roundtrip errors
x2,y2 = x2-(tempx-x1), y2-(tempy-y1)

# Add Pierre tweaks
xref,yref = x2[0]+pierrex, y2[0]+pierrey

v2ref, v3ref = mt.xytov2v3(xref,yref,'F770W')
v2ref, v3ref = v2ref[0], v3ref[0]

# Print out the new SIAF information (limit decimal places so SIAF can be exact)
print("New SIAF XDetRef, YDetRef: {0:>10.3f} ,{1:>10.3f} ".format(xref+1,yref+1))# 1-indexed
print("New SIAF V2Ref, V3Ref: {0:>10.4f} ,{1:>10.4f} ".format(v2ref,v3ref))# 1-indexed

In [ ]:
# xref, yref = siaf[thisaper].XDetRef, siaf[thisaper].YDetRef
# v2ref, v3ref = mt.xytov2v3(xref, yref, 'F770W')
# v2ref, v3ref

In [ ]:
siaf[thisaper].V2Ref, siaf[thisaper].V3Ref

In [ ]:
# xref, yref = siaf['MIRIM_MASK1065'].XDetRef, siaf['MIRIM_MASK1065'].YDetRef
# v2ref, v3ref = siaf['MIRIM_MASK1065'].V2Ref, siaf['MIRIM_MASK1065'].V3Ref
# mt_v2ref, mt_v3ref = mt.xytov2v3(xref, yref, 'F770W')

# print("Siaf XDetRef, YDetRef: \t" + f"{xref:+0.3f}, {yref:+0.3f}")
# print("Siaf V2Ref, V3Ref: \t" + f"{v2ref:+0.3f}, {v3ref:+0.3f}")
# print("MT V2Ref, V3Ref: \t" + f"{mt_v2ref[0]:+0.3f}, {mt_v3ref[0]:+0.3f}")

In [ ]:
# Add appropriate new boresight offset information to the PRD file

print("",file=open(taoutfile,"a"))
print("",file=open(scioutfile,"a"))

# Loop over each filter computing offsets
for i in range(0,nfilter):
    thisentry=siaf[thisaper]
    x,y=xref+thisbore[i].field('col_offset'),yref+thisbore[i].field('row_offset')
    v2,v3=mt.xytov2v3(x,y,'F770W')# Use F770W to avoid introducing more offsets
    dv2,dv3=v2[0]-v2ref,v3[0]-v3ref
    # Print the value of the TA filter offsets so that they can be added to the commanded position
    print("{val1:20} {val2:10} {0:<10.4f} {1:<10.4f}".format(tasign*dv2,tasign*dv3,val1=thisaper,val2=thisbore[i].field('filter')),file=open(taoutfile,"a"))
    # Print the value of the SCI offsets so that they can be added to the commanded position
    print("{val1:20} {val2:10} {0:<10.4f} {1:<10.4f}".format(scisign*dv2,scisign*dv3,val1=thisaper,val2=thisbore[i].field('filter')),file=open(scioutfile,"a"))
    if (i == nfilter-1):
        dv2_save=tasign*dv2
        dv3_save=tasign*dv3

In [ ]:
# Sanity check: if we used new SIAF and new boresight file, do we land at the correct position?
# TA will start us at xref,yref in FND
# That means the source would appear at 
xstart, ystart = xref-0.13582106901312335, yref-(-1.2279678857520966)
# OPGS adds the boresights to SIAF location to pretend source is at
v2temp,v3temp = v2ref+dv2_save, v3ref+dv3_save
# It wants to go to v2ref, v3ref and will apply an offset to get there
# Figure out what this is using any reference filter for deltas:
x1,y1=mt.v2v3toxy(v2temp,v3temp,'F770W')
x2,y2=mt.v2v3toxy(v2ref,v3ref,'F770W')
dx=x2-x1
dy=y2-y1

print('Should be at ',xref,yref)
print('Actually at ',xstart+dx[0],ystart+dy[0])

## Do the Coron1140 ##

In [ ]:
# Current SIAF location
thisaper='MIRIM_MASK1140'

# Which boresight table are we using?
thisbore=bore1140.copy()

In [ ]:
# Commissioning special offsets in Ideal coordinates
specialx=0.#2185
specialy=0.#1273

# Pierre additional tweaks in pixel coordinates
pierrex=-0.#0531
pierrey=-0.#0829

In [ ]:
# Assume OPGS did the right thing based on analysis of Visit files
# Source started at intended position in the FND filter
xold,yold=siaf[thisaper].XDetRef-1,siaf[thisaper].YDetRef-1

# According to SIAF, it would be this v2,v3 reference point
v2oldref,v3oldref=siaf[thisaper].V2Ref,siaf[thisaper].V3Ref

print('Old SIAF reference point: ',xold+1,yold+1)# 1-indexed
print('Old SIAF reference point: ',v2oldref,v3oldref)

In [ ]:
# When we swapped to Coron filter, where did it go for real?
# Subtract difference between FND and Coron filters in new measurements
xbase,ybase = xold-thisbore[14][1], yold-thisbore[14][2]

# What was the boresight shift APPLIED by OPGS?
x1, y1 = xbase-(tbdata[11][1]-tbdata[14][1]), ybase-(tbdata[11][2]-tbdata[14][2])

# Special commanding was added in Ideal coordinates, convert to dv2,dv3
# Special commanding moves the SOURCE
v2old,v3old=mt.xytov2v3(x1,y1,'F770W') # Temporary location, filter doesn't matter
angle,parity=siaf[thisaper].V3IdlYAngle,siaf[thisaper].VIdlParity
dv2sp = parity*specialx*math.cos(angle*rpd) + specialy*math.sin(angle*rpd)
dv3sp = - parity*specialx*math.sin(angle*rpd) + specialy*math.cos(angle*rpd)
# Effective location after special commanding
tempx,tempy = mt.v2v3toxy(v2old,v3old,'F770W')
x2, y2 = mt.v2v3toxy(v2old+dv2sp,v3old+dv3sp,'F770W')
# Take out roundtrip errors
x2,y2 = x2-(tempx-x1), y2-(tempy-y1)

# Add Pierre tweaks
xref,yref = x2[0]+pierrex, y2[0]+pierrey

v2ref, v3ref = mt.xytov2v3(xref,yref,'F770W')
v2ref, v3ref = v2ref[0], v3ref[0]

# Print out the new SIAF information (limit decimal places so SIAF can be exact)
print("New SIAF XDetRef, YDetRef: {0:>10.3f} ,{1:>10.3f} ".format(xref+1,yref+1))# 1-indexed
print("New SIAF V2Ref, V3Ref: {0:>10.4f} ,{1:>10.4f} ".format(v2ref,v3ref))# 1-indexed

In [ ]:
# Add appropriate new boresight offset information to the PRD file

print("",file=open(taoutfile,"a"))
print("",file=open(scioutfile,"a"))

# Loop over each filter computing offsets
for i in range(0,nfilter):
    thisentry=siaf[thisaper]
    x,y=xref+thisbore[i].field('col_offset'),yref+thisbore[i].field('row_offset')
    v2,v3=mt.xytov2v3(x,y,'F770W')# Use F770W to avoid introducing more offsets
    dv2,dv3=v2[0]-v2ref,v3[0]-v3ref
    # Print the value of the TA filter offsets so that they can be added to the commanded position
    print("{val1:20} {val2:10} {0:<10.4f} {1:<10.4f}".format(tasign*dv2,tasign*dv3,val1=thisaper,val2=thisbore[i].field('filter')),file=open(taoutfile,"a"))
    # Print the value of the SCI offsets so that they can be added to the commanded position
    print("{val1:20} {val2:10} {0:<10.4f} {1:<10.4f}".format(scisign*dv2,scisign*dv3,val1=thisaper,val2=thisbore[i].field('filter')),file=open(scioutfile,"a"))
    if (i == nfilter-1):
        dv2_save=tasign*dv2
        dv3_save=tasign*dv3

In [ ]:
# Sanity check: if we used new SIAF and new boresight file, do we land at the correct position?
# TA will start us at xref,yref in FND
# That means the source would appear at 
xstart, ystart = xref-0.3271697855783265, yref-(-0.9660751864951773)
# OPGS adds the boresights to SIAF location to pretend source is at
v2temp,v3temp = v2ref+dv2_save, v3ref+dv3_save
# It wants to go to v2ref, v3ref and will apply an offset to get there
# Figure out what this is using any reference filter for deltas:
x1,y1=mt.v2v3toxy(v2temp,v3temp,'F770W')
x2,y2=mt.v2v3toxy(v2ref,v3ref,'F770W')
dx=x2-x1
dy=y2-y1

print('Should be at ',xref,yref)
print('Actually at ',xstart+dx[0],ystart+dy[0])

## Do the Coron1550 ##

In [ ]:
# Current SIAF location
thisaper='MIRIM_MASK1550'

# Which boresight table are we using?
thisbore=bore1550.copy()

In [ ]:
# Commissioning special offsets in Ideal coordinates
specialx=0.#226
specialy=0.#156

# Pierre additional tweaks in pixel coordinates
pierrex=0.#0418
pierrey=0.#0162

In [ ]:
# Assume OPGS did the right thing based on analysis of Visit files
# Source started at intended position in the FND filter
xold,yold=siaf[thisaper].XDetRef-1,siaf[thisaper].YDetRef-1

# According to SIAF, it would be this v2,v3 reference point
v2oldref,v3oldref=siaf[thisaper].V2Ref,siaf[thisaper].V3Ref

print('Old SIAF reference point: ',xold+1,yold+1)# 1-indexed
print('Old SIAF reference point: ',v2oldref,v3oldref)

In [ ]:
# When we swapped to Coron filter, where did it go for real?
# Subtract difference between FND and Coron filters in new measurements
xbase,ybase = xold-thisbore[14][1], yold-thisbore[14][2]

# What was the boresight shift APPLIED by OPGS?
x1, y1 = xbase-(tbdata[12][1]-tbdata[14][1]), ybase-(tbdata[12][2]-tbdata[14][2])

# Special commanding was added in Ideal coordinates, convert to dv2,dv3
# Special commanding moves the SOURCE
v2old,v3old=mt.xytov2v3(x1,y1,'F770W') # Temporary location, filter doesn't matter
angle,parity=siaf[thisaper].V3IdlYAngle,siaf[thisaper].VIdlParity
dv2sp = parity*specialx*math.cos(angle*rpd) + specialy*math.sin(angle*rpd)
dv3sp = - parity*specialx*math.sin(angle*rpd) + specialy*math.cos(angle*rpd)
# Effective location after special commanding
tempx,tempy = mt.v2v3toxy(v2old,v3old,'F770W')
x2, y2 = mt.v2v3toxy(v2old+dv2sp,v3old+dv3sp,'F770W')
# Take out roundtrip errors
x2,y2 = x2-(tempx-x1), y2-(tempy-y1)

# Add Pierre tweaks
xref,yref = x2[0]+pierrex, y2[0]+pierrey

v2ref, v3ref = mt.xytov2v3(xref,yref,'F770W')
v2ref, v3ref = v2ref[0], v3ref[0]

# Print out the new SIAF information (limit decimal places so SIAF can be exact)
print("New SIAF XDetRef, YDetRef: {0:>10.3f} ,{1:>10.3f} ".format(xref+1,yref+1))# 1-indexed
print("New SIAF V2Ref, V3Ref: {0:>10.4f} ,{1:>10.4f} ".format(v2ref,v3ref))# 1-indexed

In [ ]:
# Add appropriate new boresight offset information to the PRD file

print("",file=open(taoutfile,"a"))
print("",file=open(scioutfile,"a"))

# Loop over each filter computing offsets
for i in range(0,nfilter):
    thisentry=siaf[thisaper]
    x,y=xref+thisbore[i].field('col_offset'),yref+thisbore[i].field('row_offset')
    v2,v3=mt.xytov2v3(x,y,'F770W')# Use F770W to avoid introducing more offsets
    dv2,dv3=v2[0]-v2ref,v3[0]-v3ref
    # Print the value of the TA filter offsets so that they can be added to the commanded position
    print("{val1:20} {val2:10} {0:<10.4f} {1:<10.4f}".format(tasign*dv2,tasign*dv3,val1=thisaper,val2=thisbore[i].field('filter')),file=open(taoutfile,"a"))
    # Print the value of the SCI offsets so that they can be added to the commanded position
    print("{val1:20} {val2:10} {0:<10.4f} {1:<10.4f}".format(scisign*dv2,scisign*dv3,val1=thisaper,val2=thisbore[i].field('filter')),file=open(scioutfile,"a"))
    if (i == nfilter-1):
        dv2_save=tasign*dv2
        dv3_save=tasign*dv3

In [ ]:
# Sanity check: if we used new SIAF and new boresight file, do we land at the correct position?
# TA will start us at xref,yref in FND
# That means the source would appear at 
xstart, ystart = xref-(-0.25904088927825986), yref-(-1.0810837582944424)
# OPGS adds the boresights to SIAF location to pretend source is at
v2temp,v3temp = v2ref+dv2_save, v3ref+dv3_save
# It wants to go to v2ref, v3ref and will apply an offset to get there
# Figure out what this is using any reference filter for deltas:
x1,y1=mt.v2v3toxy(v2temp,v3temp,'F770W')
x2,y2=mt.v2v3toxy(v2ref,v3ref,'F770W')
dx=x2-x1
dy=y2-y1

print('Should be at ',xref,yref)
print('Actually at ',xstart+dx[0],ystart+dy[0])

In [ ]:
thisbore

## Do the Lyot ##

In [ ]:
# Current SIAF location
thisaper='MIRIM_MASKLYOT'

# Which boresight table are we using?
thisbore=borelyot.copy()

In [ ]:
# Commissioning special offsets in Ideal coordinates
specialx=0.#147
specialy=0.#183

# Pierre additional tweaks in pixel coordinates
pierrex=0.
pierrey=0.

In [ ]:
# Assume OPGS did the right thing based on analysis of Visit files
# Source started at intended position in the FND filter
xold,yold=siaf[thisaper].XDetRef-1,siaf[thisaper].YDetRef-1

# According to SIAF, it would be this v2,v3 reference point
v2oldref,v3oldref=siaf[thisaper].V2Ref,siaf[thisaper].V3Ref

print('Old SIAF reference point: ',xold+1,yold+1)# 1-indexed
print('Old SIAF reference point: ',v2oldref,v3oldref)

In [ ]:
# When we swapped to Coron filter, where did it go for real?
# Subtract difference between FND and Coron filters in new measurements
xbase,ybase = xold-thisbore[14][1], yold-thisbore[14][2]

# What was the boresight shift APPLIED by OPGS?
x1, y1 = xbase-(tbdata[13][1]-tbdata[14][1]), ybase-(tbdata[13][2]-tbdata[14][2])

# Special commanding was added in Ideal coordinates, convert to dv2,dv3
# Special commanding moves the SOURCE
v2old,v3old=mt.xytov2v3(x1,y1,'F770W') # Temporary location, filter doesn't matter
angle,parity=siaf[thisaper].V3IdlYAngle,siaf[thisaper].VIdlParity
dv2sp = parity*specialx*math.cos(angle*rpd) + specialy*math.sin(angle*rpd)
dv3sp = - parity*specialx*math.sin(angle*rpd) + specialy*math.cos(angle*rpd)
# Effective location after special commanding
tempx,tempy = mt.v2v3toxy(v2old,v3old,'F770W')
x2, y2 = mt.v2v3toxy(v2old+dv2sp,v3old+dv3sp,'F770W')
# Take out roundtrip errors
x2,y2 = x2-(tempx-x1), y2-(tempy-y1)

# Add Pierre tweaks
xref,yref = x2[0]+pierrex, y2[0]+pierrey

v2ref, v3ref = mt.xytov2v3(xref,yref,'F770W')
v2ref, v3ref = v2ref[0], v3ref[0]

# Print out the new SIAF information (limit decimal places so SIAF can be exact)
print("New SIAF XDetRef, YDetRef: {0:>10.3f} ,{1:>10.3f} ".format(xref+1,yref+1))# 1-indexed
print("New SIAF V2Ref, V3Ref: {0:>10.4f} ,{1:>10.4f} ".format(v2ref,v3ref))# 1-indexed

In [ ]:
# Add appropriate new boresight offset information to the PRD file

print("",file=open(taoutfile,"a"))
print("",file=open(scioutfile,"a"))

# Loop over each filter computing offsets
for i in range(0,nfilter):
    thisentry=siaf[thisaper]
    x,y=xref+thisbore[i].field('col_offset'),yref+thisbore[i].field('row_offset')
    v2,v3=mt.xytov2v3(x,y,'F770W')# Use F770W to avoid introducing more offsets
    dv2,dv3=v2[0]-v2ref,v3[0]-v3ref
    # Print the value of the TA filter offsets so that they can be added to the commanded position
    print("{val1:20} {val2:10} {0:<10.4f} {1:<10.4f}".format(tasign*dv2,tasign*dv3,val1=thisaper,val2=thisbore[i].field('filter')),file=open(taoutfile,"a"))
    # Print the value of the SCI offsets so that they can be added to the commanded position
    print("{val1:20} {val2:10} {0:<10.4f} {1:<10.4f}".format(scisign*dv2,scisign*dv3,val1=thisaper,val2=thisbore[i].field('filter')),file=open(scioutfile,"a"))
    if (i == nfilter-1):
        dv2_save=tasign*dv2
        dv3_save=tasign*dv3

In [ ]:
# Sanity check: if we used new SIAF and new boresight file, do we land at the correct position?
# TA will start us at xref,yref in FND
# That means the source would appear at 
xstart, ystart = xref-(0.9187159536913048), yref-(-1.5764380715635262)
# OPGS adds the boresights to SIAF location to pretend source is at
v2temp,v3temp = v2ref+dv2_save, v3ref+dv3_save
# It wants to go to v2ref, v3ref and will apply an offset to get there
# Figure out what this is using any reference filter for deltas:
x1,y1=mt.v2v3toxy(v2temp,v3temp,'F770W')
x2,y2=mt.v2v3toxy(v2ref,v3ref,'F770W')
dx=x2-x1
dy=y2-y1

print('Should be at ',xref,yref)
print('Actually at ',xstart+dx[0],ystart+dy[0])

<BR>

## Do the imaging regions ## 

In [ ]:
apernames=['MIRIM_FULL','MIRIM_ILLUM','MIRIM_BRIGHTSKY','MIRIM_SUB256','MIRIM_SUB128','MIRIM_SUB64']
naper=len(apernames)

In [ ]:
# Loop over each aperture
for j in range(0,naper):
    print("",file=open(taoutfile,"a"))
    print("",file=open(scioutfile,"a"))

    # Loop over each filter computing offsets
    for i in range(0,nfilter):
        thisentry=siaf[apernames[j]]
        xdet,ydet=thisentry.XDetRef-1,thisentry.YDetRef-1 # Define the location in 0-indexed pixels (subtract 1 from SIAF)
        v2ref,v3ref=thisentry.V2Ref,thisentry.V3Ref
        x,y=xdet+tbdata[i].field('col_offset'),ydet+tbdata[i].field('row_offset')
        v2,v3=model(x,y)
        dv2,dv3=v2-v2ref,v3-v3ref
        # Print the value of the TA filter offsets so that they can be added to the commanded position
        print("{val1:20} {val2:10} {0:<10.4f} {1:<10.4f}".format(tasign*dv2,tasign*dv3,val1=apernames[j],val2=tbdata[i].field('filter')),file=open(taoutfile,"a"))
        # Print the value of the SCI offsets so that they can be added to the commanded position
        print("{val1:20} {val2:10} {0:<10.4f} {1:<10.4f}".format(scisign*dv2,scisign*dv3,val1=apernames[j],val2=tbdata[i].field('filter')),file=open(scioutfile,"a"))
    